<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Querying-for-Interviews-with-Person-X" data-toc-modified-id="Querying-for-Interviews-with-Person-X-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Querying for Interviews with Person X</a></span><ul class="toc-item"><li><span><a href="#Interviews-with-Bernie-Sanders" data-toc-modified-id="Interviews-with-Bernie-Sanders-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Interviews with Bernie Sanders</a></span></li><li><span><a href="#Interviews-with-Kellyanne-Conway-and-John-McCain" data-toc-modified-id="Interviews-with-Kellyanne-Conway-and-John-McCain-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Interviews with Kellyanne Conway and John McCain</a></span></li><li><span><a href="#Modify-after-detecting-fixed-stride-faces-and-new-identity-labeling-model" data-toc-modified-id="Modify-after-detecting-fixed-stride-faces-and-new-identity-labeling-model-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Modify after detecting fixed stride faces and new identity labeling model</a></span></li></ul></li><li><span><a href="#Now-let's-try-to-detect-interviews-in-the-whole-dataset" data-toc-modified-id="Now-let's-try-to-detect-interviews-in-the-whole-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Now let's try to detect interviews in the whole dataset</a></span><ul class="toc-item"><li><span><a href="#Detect-all-the-interviews-with-2016-presidential-candidates" data-toc-modified-id="Detect-all-the-interviews-with-2016-presidential-candidates-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Detect all the interviews with 2016 presidential candidates</a></span></li><li><span><a href="#Visulaize-result" data-toc-modified-id="Visulaize-result-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Visulaize result</a></span></li><li><span><a href="#Analyze-result-stats-(create-csv-for-tablao)" data-toc-modified-id="Analyze-result-stats-(create-csv-for-tablao)-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Analyze result stats (create csv for tablao)</a></span></li><li><span><a href="#Interview-montage" data-toc-modified-id="Interview-montage-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Interview montage</a></span></li><li><span><a href="#Interview-validation" data-toc-modified-id="Interview-validation-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Interview validation</a></span><ul class="toc-item"><li><span><a href="#Create-gt-from-csv" data-toc-modified-id="Create-gt-from-csv-2.5.1"><span class="toc-item-num">2.5.1&nbsp;&nbsp;</span>Create gt from csv</a></span></li><li><span><a href="#Validate-interview-of-person-X" data-toc-modified-id="Validate-interview-of-person-X-2.5.2"><span class="toc-item-num">2.5.2&nbsp;&nbsp;</span>Validate interview of person X</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
# Imports. Run this first!

from query.models import LabeledInterview, LabeledPanel, LabeledCommercial, Video, FaceIdentity, Face
from esper.rekall import *
from rekall.temporal_predicates import *
from rekall.spatial_predicates import *
from rekall.interval_list import IntervalList
from esper.prelude import esper_widget
#from esper.captions import topic_search
from django.db.models import FloatField

sandbox_videos = [529, 763, 2648, 3459, 3730, 3769, 3952, 4143, 4611, 5281, 6185, 7262, 8220,
    8697, 8859, 9215, 9480, 9499, 9901, 10323, 10335, 11003, 11555, 11579, 11792,
    12837, 13058, 13141, 13247, 13556, 13827, 13927, 13993, 14482, 15916, 16215,
    16542, 16693, 16879, 17458, 17983, 19882, 19959, 20380, 20450, 23181, 23184,
    24193, 24847, 24992, 25463, 26386, 27188, 27410, 29001, 31378, 32472, 32996,
    33004, 33387, 33541, 33800, 34359, 34642, 36755, 37107, 37113, 37170, 38275,
    38420, 40203, 40856, 41480, 41725, 42756, 45472, 45645, 45655, 45698, 48140,
    49225, 49931, 50164, 50561, 51175, 52075, 52749, 52945, 53355, 53684, 54377,
    55711, 57384, 57592, 57708, 57804, 57990, 59122, 59398, 60186]

# Querying for Interviews with Person X

We have an annotated sandbox of panels, interviews, and commercials (`query_labeledpanel`, `query_labeledinterview`, `query_commercial`). In this notebook we'll try to use write queries for these concepts and compare the results against our labels.

A few notes about the panels:
* Panel segments go from the introduction of panelists to the shot where the host says "thank you" or "goodbye" to the panelists. Sometimes the camera will cut wide and show all the panelists before a commercial break; such shots are *not* included in the labeled segments.
* Panel segments are split up by commercials (i.e., if the same panel appears before and after a commercial break, that will be two panel segments).
* Panels segments do not include segments where the host is just cutting to multiple reporters out in the field to cover some news story.

A few notes about the interviews. See the third bullet point in particular.
* Interview segments go from the first shot of the guest to where the host thanks the guest. Sometimes the host thanks the guest while the guest is on screen, and sometimes the host thanks the guest off-screen. In the former case, the segment continues until the guest is no longer on screen; in the latter case, the segment stops when the host changes the subject after thanking the guest.
* Sometimes the host doesn't thank the guest; in this case, the segment ends when the guest is no longer on screen or when the host changes the subject.
* **Interviews with analysts and correspondents from the same network are *not* included. This is to differentiate between the typical interview and "interviews" where the guest is just presenting a news segment.**
* Interviews with reporters from the same network are also usually not included, *unless* the format of the interview is sufficiently different from the typical "here's a reporter to tell you the news" format. This is a judgment call on my (Dan Fu) part.

This dataset also includes extra annotation of interviews with Kellyanne Conway, Bernie Sanders, and John McCain.
* These interview segments include **any** segment where Kellyanne Conway, Bernie Sanders, or John McCain appear and are being interviewed. This includes segments during commercials or short clips where an interview of one of them is being played on another channel or show.
* These segments also include clips where the guest appears for a few seconds as a "preview" before a commercial break.
* Each of these segments is annotated with the name of the guest(s) and interviewer(s).

A few notes about the commercials.
* Commercial segments go from the beginning of the commercial break to the end of the commercial break. Sometimes networks will put in a segment from the network to let the viewers know that the commercial break is ending (think "this is CNN, the must trusted name in news" segment). Such segments are included in the commercial segments.

Let's define some statistics. Suppose we have a set of intervals `query` that represent our query, and a set of intervals `ground_truth` that represent our ground truth. We are interested in four statistics:
* **precision**: This is your standard definition of precision, computed over the intervals: `sum(overlap(query, ground_truth)) / sum(query)`
* **recall**: This is your standard definition of recall, computed over the intervals: `sum(overlap(query, ground_truth)) / sum(ground_truth)`
* **precision_per_item**: We may also be interested in *how many* segments we hit. How many segments in `query` overlap with *any* segment in `ground_truth`? This is `sum(count(overlap(query, ground_truth))) / sum(count(query))`.
* **recall_per_item**: Similar to precision_per_item. How many segments in `ground_truth` overlap with *any* segment in `query`? `sum(count(overlap(query, ground_truth))) / sum(count(ground_truth))`.

In [ ]:
# Returns precision, recall, precision_per_item, recall_per_item
def compute_statistics(query_intrvllists, ground_truth_intrvllists):
    total_query_time = 0
    total_query_segments = 0
    total_ground_truth_time = 0
    total_ground_truth_segments = 0
    
    for video in query_intrvllists:
        total_query_time += query_intrvllists[video].coalesce().get_total_time()
        total_query_segments += query_intrvllists[video].size()
    for video in ground_truth_intrvllists:
        total_ground_truth_time += ground_truth_intrvllists[video].coalesce().get_total_time()
        total_ground_truth_segments += ground_truth_intrvllists[video].size()
        
    total_overlap_time = 0
    overlapping_query_segments = 0
    overlapping_ground_truth_segments = 0
    
    for video in query_intrvllists:
        if video in ground_truth_intrvllists:
            query_list = query_intrvllists[video]
            gt_list = ground_truth_intrvllists[video]
            
            total_overlap_time += query_list.overlaps(gt_list).coalesce().get_total_time()
            overlapping_query_segments += query_list.filter_against(gt_list, predicate=overlaps()).size()
            overlapping_ground_truth_segments += gt_list.filter_against(query_list, predicate=overlaps()).size()
    
    if total_query_time == 0:
        precision = 1.0
        precision_per_item = 1.0
    else:
        precision = total_overlap_time / total_query_time
        precision_per_item = overlapping_query_segments / total_query_segments
    
    if total_ground_truth_time == 0:
        recall = 1.0
        recall_per_item = 1.0
    else:
        recall = total_overlap_time / total_ground_truth_time
        recall_per_item = overlapping_ground_truth_segments / total_ground_truth_segments
    
    return precision, recall, precision_per_item, recall_per_item

def print_statistics(query_intrvllists, ground_truth_intrvllists):
    precision, recall, precision_per_item, recall_per_item = compute_statistics(
        query_intrvllists, ground_truth_intrvllists)

    print("Precision: ", precision)
    print("Recall: ", recall)
    print("Precision Per Item: ", precision_per_item)
    print("Recall Per Item: ", recall_per_item)

First, let's just visualize all the labeled data. Interviews are in red, panels are in blue, and commercials are in purple.

In [ ]:
interviews = LabeledInterview.objects \
        .annotate(fps=F('video__fps')) \
        .annotate(min_frame=F('fps') * F('start')) \
        .annotate(max_frame=F('fps') * F('end'))
panels = LabeledPanel.objects \
        .annotate(fps=F('video__fps')) \
        .annotate(min_frame=F('fps') * F('start')) \
        .annotate(max_frame=F('fps') * F('end'))
commercials = LabeledCommercial.objects \
        .annotate(fps=F('video__fps')) \
        .annotate(min_frame=F('fps') * F('start')) \
        .annotate(max_frame=F('fps') * F('end'))

result = intrvllists_to_result(qs_to_intrvllists(interviews))
add_intrvllists_to_result(result, qs_to_intrvllists(panels), color="blue")
add_intrvllists_to_result(result, qs_to_intrvllists(commercials), color="purple")

esper_widget(result)

## Interviews with Bernie Sanders

In [ ]:
# Let's get all interviews of Bernie Sanders in our dataset and display it as black.
# For this task, we won't display any interviews that weren't original appearances.

bernie_interviews = LabeledInterview.objects \
        .annotate(fps=F('video__fps')) \
        .annotate(min_frame=F('fps') * F('start')) \
        .annotate(max_frame=F('fps') * F('end')) \
        .filter(guest1="bernie sanders")

bernie_interviews_intrvllists = qs_to_intrvllists(bernie_interviews)
bernie_interviews_original_intrvllists = qs_to_intrvllists(bernie_interviews.filter(original=True))

# Hide result in a function for namespace reasons
def get_result():
    result = intrvllists_to_result(bernie_interviews_original_intrvllists, color='black')

    return result

esper_widget(get_result(), show_middle_frame=True)

In [ ]:
# Helper function to get results with ground truth

def result_with_ground_truth(intrvllists):
    result = intrvllists_to_result(bernie_interviews_original_intrvllists, color='black')
    add_intrvllists_to_result(result, intrvllists, color='red')
    return result

In [ ]:
# Let's query for Bernie Sanders interviews. This may take a while to materialize all the data.

identities = FaceIdentity.objects.filter(face__shot__video_id__in=sandbox_videos)
hosts = identities.filter(face__is_host=True)
sanders = identities.filter(identity__name="bernie sanders").filter(probability__gt=0.7)

hosts_intrvllists = qs_to_intrvllists(hosts
    .annotate(video_id=F("face__shot__video_id"))
    .annotate(min_frame=F("face__shot__min_frame"))
    .annotate(max_frame=F("face__shot__max_frame")))
sanders_intrvllists = qs_to_intrvllists(sanders
    .annotate(video_id=F("face__shot__video_id"))
    .annotate(min_frame=F("face__shot__min_frame"))
    .annotate(max_frame=F("face__shot__max_frame")))

In [ ]:
# Get all shots with Bernie Sanders and a host
sanders_with_host_intrvllists = {}
for video in sanders_intrvllists:
    if video in hosts_intrvllists:
        sanders_with_host_intrvllists[video] = sanders_intrvllists[video].overlaps(hosts_intrvllists[video]).coalesce()

print_statistics(sanders_with_host_intrvllists, bernie_interviews_original_intrvllists)

esper_widget(result_with_ground_truth(sanders_with_host_intrvllists))

What do we get from those statistics? We are missing half the interviews, but we're hitting all of them. This tells us that part of our problem has to do with not coalescing well enough. We also have a problem where half our query segments do *not* hit an interview, so we need to cull some. Let's try something else.

In [ ]:
'''
We're going to look for the following patterns:
    (Bernie Sanders + host) -> host OR
    host -> (Bernie Sanders + host) OR
    (Bernie Sanders + host) -> Bernie Sanders OR
    Bernie Sanders -> (Bernie Sanders + host)

We'll coalesce that, and then check in with the Esper widget again.
'''
sanders_interview_intrvllists = {}
for video in sanders_with_host_intrvllists:
    sanders_with_host = sanders_with_host_intrvllists[video]
    hosts = hosts_intrvllists[video]
    sanders = sanders_intrvllists[video]
    
    sanders_interview_intrvllists[video] = sanders_with_host.merge(
        hosts, predicate=or_pred(before(max_dist=10), after(max_dist=10))).set_union(
        sanders_with_host.merge(sanders, predicate=or_pred(before(max_dist=10), after(max_dist=10)))
    ).coalesce()

print_statistics(sanders_interview_intrvllists, bernie_interviews_original_intrvllists)

esper_widget(result_with_ground_truth(sanders_interview_intrvllists))

We're much closer to getting all the interviews, but we still have some large gaps. Let's try to see what's going on.

In [ ]:
investigation_result = intrvllists_to_result(bernie_interviews_original_intrvllists, color='black')
add_intrvllists_to_result(investigation_result, sanders_with_host_intrvllists, color='orange')
add_intrvllists_to_result(investigation_result, sanders_intrvllists, color='blue')
add_intrvllists_to_result(investigation_result, sanders_interview_intrvllists, color='red')

esper_widget(investigation_result)

There are some gaps because of consecutive Bernie Sanders or host shots. Let's dilate and coalesce those and have another go at that.

In [ ]:
sanders_interview_consec_intrvllists = {}
for video in sanders_with_host_intrvllists:
    sanders_with_host = sanders_with_host_intrvllists[video]
    hosts = hosts_intrvllists[video].dilate(10).coalesce().dilate(-10)
    sanders = sanders_intrvllists[video].dilate(10).coalesce().dilate(-10)
    
    sanders_interview_consec_intrvllists[video] = sanders_with_host.merge(
        hosts, predicate=or_pred(or_pred(overlaps(), before(max_dist=10)), after(max_dist=10))).set_union(
        sanders_with_host.merge(sanders, predicate=or_pred(or_pred(overlaps(), before(max_dist=10)), after(max_dist=10)))
    ).coalesce()

print_statistics(sanders_interview_consec_intrvllists, bernie_interviews_original_intrvllists)

esper_widget(result_with_ground_truth(sanders_interview_consec_intrvllists))

In [ ]:
sanders_interview_filtered_intrvllists = {}
for video in sanders_interview_intrvllists:
    sanders_interview = sanders_interview_consec_intrvllists[video]
    
    sanders_interview_filtered_intrvllists[video] = sanders_interview \
        .dilate(600) \
        .coalesce() \
        .dilate(-600) \
        .filter_length(min_length=1350)

print_statistics(sanders_interview_filtered_intrvllists, bernie_interviews_original_intrvllists)

esper_widget(result_with_ground_truth(sanders_interview_filtered_intrvllists))

That looks pretty good. We still have some false positives, but it's hard to get rid of those with what we have right now. Let's summarize what we did:

In [ ]:
# Show multiple stages of our query process all in one timeline.
summarize_bernie_result = intrvllists_to_result(bernie_interviews_original_intrvllists, color='black')
add_intrvllists_to_result(summarize_bernie_result, sanders_with_host_intrvllists, color='orange')
add_intrvllists_to_result(summarize_bernie_result, sanders_intrvllists, color='blue')
add_intrvllists_to_result(summarize_bernie_result, hosts_intrvllists, color='purple')
add_intrvllists_to_result(summarize_bernie_result, sanders_interview_intrvllists, color='green')
add_intrvllists_to_result(summarize_bernie_result, sanders_interview_consec_intrvllists, color='brown')
add_intrvllists_to_result(summarize_bernie_result, sanders_interview_filtered_intrvllists, color='red')

esper_widget(summarize_bernie_result)

## Interviews with Kellyanne Conway and John McCain

Now that we have a simple query for interviews with Bernie Sanders, let's do the same thing for Kellyanne Conway and John McCain, using our best method from before.

In [ ]:
# ground truth for interviews where guest 1 is X
def ground_truth_interviews_intrvllists(name, original=True):
    interviews = LabeledInterview.objects \
        .annotate(fps=F('video__fps')) \
        .annotate(min_frame=F('fps') * F('start')) \
        .annotate(max_frame=F('fps') * F('end')) \
        .filter(guest1=name)
    if original:
        interviews = interviews.filter(original=original)
    return qs_to_intrvllists(interviews)

# intrvllists for shots with a face with identity X
def named_person_intrvllists(name):
    identities = FaceIdentity.objects.filter(face__shot__video_id__in=sandbox_videos)
    person = identities.filter(identity__name=name).filter(probability__gt=0.7)
    
    return qs_to_intrvllists(person
        .annotate(video_id=F("face__shot__video_id"))
        .annotate(min_frame=F("face__shot__min_frame"))
        .annotate(max_frame=F("face__shot__max_frame")))

# helper function to get hosts
def host_intrvllists():
    host = identities.filter(face__is_host=True)

    return qs_to_intrvllists(hosts
        .annotate(video_id=F("face__shot__video_id"))
        .annotate(min_frame=F("face__shot__min_frame"))
        .annotate(max_frame=F("face__shot__max_frame")))

# query for interviews of person X
def interview_query(person_intrvllists, host_intrvllists):
    interview_intrvllists = {}
    for video in person_intrvllists:
        if video not in host_intrvllists:
            continue
        person = person_intrvllists[video]
        host = host_intrvllists[video]
        person_with_host = person.overlaps(host).coalesce()
        
        overlaps_before_or_after_pred = or_pred(or_pred(
            overlaps(), before(max_dist=10)), after(max_dist=10))
        
        interview_candidates = person_with_host \
            .merge(host, predicate=overlaps_before_or_after_pred) \
            .set_union(person_with_host.merge(
                person, predicate=overlaps_before_or_after_pred)) \
            .coalesce()
        
        interviews_filtered = interview_candidates \
            .dilate(600) \
            .coalesce() \
            .dilate(-600) \
            .filter_length(min_length=1350)
        
        if interviews_filtered.size() > 0:
            interview_intrvllists[video] = interviews_filtered
    
    return interview_intrvllists

In [ ]:
# Helper function to do all the above in one call

def summarize_named_interview(name, original=True):
    gt = ground_truth_interviews_intrvllists(name, original)
    person = named_person_intrvllists(name)
    person_interviews = interview_query(person, hosts_intrvllists)

    print_statistics(person_interviews, gt)

    summarize_result = intrvllists_to_result(gt, color='black')
    add_intrvllists_to_result(summarize_result, person, color='blue')
    add_intrvllists_to_result(summarize_result, hosts_intrvllists, color='purple')
    add_intrvllists_to_result(summarize_result, person_interviews, color='red')

    return summarize_result

In [ ]:
# This will take a while to materialize some of the data

kellyanne_result = summarize_named_interview("kellyanne conway")
esper_widget(kellyanne_result)

In [ ]:
# This will take a while to materialize some of the data

mccain_result = summarize_named_interview("john mccain")
esper_widget(mccain_result)

So in summary, the precision for Kellyanne Conway is pretty good (95%), but it's not as good for John McCain. The main reason is that there are very few interviews of John McCain, so any false positives (there is one) throw the precision numbers quite off. There's also a big false negative for John McCain - an interview with Jake Tapper that was played verbatim on CNN, but not on Jake Tapper's show. This false negative occurs because Jake Tapper's face wasn't registered as a host during that playtime.

## Modify after detecting fixed stride faces and new identity labeling model 

In [ ]:
from esper.interview import *

In [ ]:
def test_interview(person_name):
    if person_name == 'donald trump':
        gt = pickle.load(open('/app/data/interview_gt.pkl', 'rb'))
        gt = VideoIntervalCollection({id: IntervalList(list) for id, list in gt.items()})
        gt = intrvlcol_second2frame(gt)
    else:
        gt = ground_truth_interviews(person_name)
    
    video_ids=list(gt.get_allintervals().keys())
#     print(video_ids)

    host_list = [h.name for s in CanonicalShow.objects.all() for h in s.hosts.all() ]
    host_list = list(set(host_list))
    
    person_intrvlcol, host_intrvlcol, commercial = load_intervals(video_ids, person_name, host_list)
#     return person_intrvlcol, host_intrvlcol, commercial, gt
    
    interviews, person_only, host_only, person_with_host = interview_query(person_intrvlcol, host_intrvlcol, commercial)
#     return interviews, person_in_interviews
    
    print("Total length: %.1fh" % (count_duration(intrvlcol_frame2second(gt)) / 3600))
    print_statistics(intrvlcol_second2frame(interviews).get_allintervals(), gt.get_allintervals())

    summarize_result = intrvllists_to_result(gt.get_allintervals(), color='green')
    add_intrvllists_to_result(summarize_result, intrvlcol_second2frame(commercial).get_allintervals(), color='gray')
    add_intrvllists_to_result(summarize_result, intrvlcol_second2frame(person_intrvlcol).get_allintervals(), color='blue')
    add_intrvllists_to_result(summarize_result, intrvlcol_second2frame(host_intrvlcol).get_allintervals(), color='purple')
    add_intrvllists_to_result(summarize_result, intrvlcol_second2frame(person_with_host).get_allintervals(), color='orange')
    add_intrvllists_to_result(summarize_result, intrvlcol_second2frame(interviews).get_allintervals(), color='red')
    return summarize_result

In [ ]:
summarize_result_b = test_interview('bernie sanders')
# esper_widget(summarize_result_b)

Total: 3.5h
Precision:  0.9171231330081376  
Recall:  0.9754783719638195  
Precision Per Item:  1.0  
Recall Per Item:  0.8666666666666667  

In [ ]:
summarize_result_k = test_interview('kellyanne conway')
# esper_widget(summarize_result_k)

Total: 2.2
Precision:  0.9177643900547426  
Recall:  0.890869917822613  
Precision Per Item:  1.0  
Recall Per Item:  0.75  

In [ ]:
summarize_result_j = test_interview('john mccain')
# esper_widget(summarize_result_j)

Total: 0.9
Precision:  0.8590926781804915  
Recall:  0.9953915876457966  
Precision Per Item:  1.0  
Recall Per Item:  0.875  

# Now let's try to detect interviews in the whole dataset 

In [ ]:
from esper.interview import *

## Detect all the interviews with 2016 presidential candidates from 2015 to 2017

In [ ]:
# 2016 presidential candidates
guest_list = ['Barack Obama', 'Donald Trump', 'Ted Cruz', 'John Kasich', 'Marco Rubio', 'Ben Carson', 'Jeb Bush',
'Jim Gilmore', 'Chris Christie', 'Carly Fiorina', 'Rick Santorum', 'Rand Paul', 'Mike Huckabee',
'Hillary Clinton', 'Bernie Sanders', 'Lincoln Chafee', 'Martin O’Malley', 'Jim Webb',
]

In [ ]:
# get host list from canonical shows
canonical_show_list = [s.name for s in CanonicalShow.objects.all()]
host_list = [h.name for s in CanonicalShow.objects.all() for h in s.hosts.all() ]
host_list = list(set(host_list))
len(host_list)

In [ ]:
# get video list
videos = Video.objects.filter(time__year=2015, duplicate=False, corrupted=False,
                             show__name__in=canonical_show_list)
video_ids = [video.id for video in videos]
len(video_ids)

In [ ]:
# get all host and commercial
host_intrvlcol = get_person_intrvlcol(host_list, video_ids=video_ids, face_size=0.2, \
                                          stride_face=False, probability=0.7, granularity='second')
commercial = get_commercial_intrvlcol(video_ids, granularity='second')

In [ ]:
for person_name in guest_list:
    print('Searching interviews for {} ...'.format(person_name))
    person_intrvlcol = get_person_intrvlcol(person_name, video_ids=video_ids, face_size=0.2, 
                                            stride_face=False, probability=0.7, granularity='second')
    
    interviews, person_in_interviews = interview_query(person_intrvlcol, host_intrvlcol, commercial)
    save_interview(person_name, interviews, person_in_interviews)    

## Detect all the interviews with 40 important political figures in 10 years

In [ ]:
# political figures
guest_list = ['Barack Obama', 'Donald Trump', 'Ted Cruz', 'John Kasich', 'Marco Rubio', 'Ben Carson', 'Jeb Bush',
'Jim Gilmore', 'Chris Christie', 'Carly Fiorina', 'Rick Santorum', 'Rand Paul', 'Mike Huckabee',
'Hillary Clinton', 'Bernie Sanders', 'Lincoln Chafee', 'Martin O’Malley', 'Jim Webb',

'Sarah Palin', 'John Boehner', 'Paul Ryan', 'Newt Gingrich','Nancy Pelosi','Elizabeth Warren', 'Mitch McConnell',
'Chuck Schumer','Harry Reid','Joe Biden', 'Kevin McCarthy', 'Steve Scalise', 'Bobby Jindal', 'John Cornyn',
'Dick Durbin','Orrin Hatch', 'Lindsey Graham', 'Mitt Romney', 'Michelle Obama' ,'Bill Clinton', 
'George W Bush', 'Tim Kaine' ]

In [ ]:
# get host list from canonical shows
canonical_show_list = [s.name for s in CanonicalShow.objects.all()]
host_list = [h.name for s in CanonicalShow.objects.all() for h in s.hosts.all() ]
host_list = list(set(host_list))
len(host_list)

In [ ]:
guest_ids = [Identity.objects.filter(name=name.lower())[0].id for name in guest_list]
host_ids = [Identity.objects.filter(name=name.lower())[0].id for name in host_list]
print(len(guest_ids), len(host_ids))

In [ ]:
# load all identity intervals precomputed using rust
identity_dict = pickle.load(open('/app/data/identity_intervals_rust.pkl', 'rb'))

In [ ]:
# get all host 
host_intrvllists = {}
for host_id in host_ids:
    if not host_id in identity_dict:
        continue
    print(host_id)
    for video_id, values in identity_dict[host_id].items():
        if not video_id in host_intrvllists:
            host_intrvllists[video_id] = []
        for stime, height in values:
            host_intrvllists[video_id].append((stime, stime + 1./30, host_id))

In [ ]:
# dilate each frame interval to the length of sample rate
SAMPLE_RATE = 3
cnt = 0
for video_id, intervals in host_intrvllists.items():
    host_intrvllists[video_id] = IntervalList(intervals).dilate(SAMPLE_RATE/2).coalesce().dilate(-SAMPLE_RATE/2)
    cnt += 1
    if cnt % 1000 == 0:
        print(cnt)
host_intrvlcol = VideoIntervalCollection(host_intrvllists)

In [ ]:
# get all commercial
video_ids = host_intrvlcol.get_allintervals().keys()
commercial = get_commercial_intrvlcol(video_ids, granularity='second')

In [ ]:
def get_person_intrvlcol_from_rust(person_id):
    person_intrvlcol = {}
    cnt = 0
    for video_id, values in identity_dict[person_id].items():
        intervals = []
        for stime, height in values:
            intervals.append((stime, stime+1./30, height))
        person_intrvlcol[video_id] = IntervalList(intervals).dilate(SAMPLE_RATE/2).coalesce().dilate(-SAMPLE_RATE/2)
        cnt += 1
        if cnt % 1000 == 0:
            print(cnt)
    return VideoIntervalCollection(person_intrvlcol)

In [ ]:
for person_id, person_name in zip(guest_ids, guest_list):
    print(person_id, person_name)
    person_intrvlcol = get_person_intrvlcol_from_rust(person_id)
    print("Get {} videos for {}".format(len(person_intrvlcol.get_allintervals()), person_name))
    
    interviews, person_only, host_only, person_with_host = interview_query(person_intrvlcol, host_intrvlcol, commercial)
    print("Get {} interview segments for {}".format(count_intervals(interviews), person_name))
    save_interview(person_name, '10y', interviews, person_only, host_only, person_with_host) 

## Visulaize result

In [ ]:
interviews, _, _, _ = load_interview("Newt Gingrich", suffix='10y2')
print("Num of videos %d, Num of intervals %d, Total length %.1fh" %
    (len(interviews.get_allintervals()), count_intervals(interviews), count_duration(interviews) / 3600))

In [ ]:
esper_widget(intrvllists_to_result(intrvlcol_second2frame(interviews).get_allintervals()))

## Analyze result stats (create csv for tablao) 

In [ ]:
csv = open('/app/result/interview/interview-10y-all-v2.csv', 'w')
csv.write(','.join(['person_name', 'video_id', 'date', 'channel', 'host_name', 'host_ratio', 'host_rank'\
                    'interview_time', 'person_time', 'host_time', 'person_host_time']) + '\n')

ident_id_to_name = {x.id: x.name for x in Identity.objects.all()}
channel_name_to_hosts = {}
for v in Video.objects.all().distinct('show__canonical_show'):
    canon_show = v.show.canonical_show
    if v.channel.name not in channel_name_to_hosts:
        channel_name_to_hosts[v.channel.name] = set()
    channel_name_to_hosts[v.channel.name].update([h.name for h in canon_show.hosts.all()])
labeler_ids = {l.id for l in Labeler.objects.filter(name__contains='face-identity-converted:')} | {l.id for l in Labeler.objects.filter(name__contains='face-identity:')}

for person_name in guest_list:
    print(person_name)
    interview, person_only, host_only, person_with_host = load_interview(person_name, suffix='10y2')

    for video_id in interview.get_allintervals():
        video = Video.objects.filter(id=video_id)[0]
        date = video.time.strftime('%Y-%m-%d')        
        channel = video.channel.name
        # get host from show name
#         show = Show.objects.filter(name=video.show.canonical_show.name)[0]
#         if len(show.hosts.all()) > 0:
#             host_name = show.hosts.all()[0].name
#         else:
#             host_name = ''
        
        # get host by finding the most frequent person not X
        ident_cnt = {}
        for i in interview.get_intervallist(video_id).get_intervals():
            idents = FaceIdentity.objects.filter(
                face__frame__video_id=video_id,
                face__frame__number__gte=i.start * video.fps, 
                face__frame__number__lte=i.end * video.fps,
                probability__gt=0.8,
                labeler__id__in=labeler_ids
                )
            for i in idents:
                ident_name = i.identity.name
                is_host = ident_name in channel_name_to_hosts[channel]
                if is_host and ident_name != person_name.lower():
                    if not ident_name in ident_cnt:
                        ident_cnt[ident_name] = 0
                    ident_cnt[ident_name] += 1
#         print(ident_cnt)
        ident_list = [(value, key) for key, value in ident_cnt.items()]
        ident_list.sort()
        ident_list = ident_list[::-1]
        cnt_sum = sum([value for key, value in ident_cnt.items()])
        ident_list = [(ident_name, 1.*cnt/cnt_sum) for (cnt, ident_name) in ident_list]
#         print(ident_list)
        
        interview_time = count_duration(interview.get_intervallist(video_id))
        person_time = count_duration(person_only.get_intervallist(video_id))
        host_time = count_duration(host_only.get_intervallist(video_id))
        person_host_time = count_duration(person_with_host.get_intervallist(video_id))
        
        for rank, (ident_name, ratio) in enumerate(ident_list):
            if rank > 3 or ratio < 0.1:
                continue
            line = ','.join([person_name.lower(), str(video_id), date, channel, \
                             ident_name, str(ratio), str(rank), \
                            str(interview_time), str(person_time), str(host_time), str(person_host_time)]) + '\n'
            csv.write(line)
        

## Interview montage

In [ ]:
person_name = 'Ted Cruz'
montage = montage_interview(person_name, suffix='final', width=2160, num_cols=10, target_height = 2160 // 10 * 9 // 16)

In [ ]:
import cv2
import matplotlib.pyplot as plt
# plt.imshow(cv2.cvtColor(montage, cv2.COLOR_BGR2RGB))
cv2.imwrite('/app/result/interview/{}-montage.jpg'.format(person_name), montage)

In [ ]:
shotcut_interview(person_name)

## Interview validation

### Create gt from csv

In [ ]:
def time2second(time):
    return time[0]*3600 + time[1]*60 + time[2]

csv = open('/app/data/interview_groundtruth.csv', 'r')
interview_gt = {}
for line in csv:
    columns = line[:-1].split(',')
    video_path = columns[0].replace('tvnews_videos_', '')
    video = Video.objects.filter(path__contains=video_path)[0]
    
    interview_gt[video.id] = []
    for i in range(1, len(columns)):
        if columns[i] == '':
            continue
        span = columns[i].split('-')
        start = span[0].split(':')
        end = span[1].split(':')
        start = time2second((int(start[0]), int(start[1]), int(start[2])))
        end = time2second((int(end[0]), int(end[1]), int(end[2])))
        interview_gt[video.id].append((start, end, 0))
interview_gt

In [ ]:
pickle.dump(interview_gt, open('/app/data/interview_gt.pkl', 'wb'))

### Validate interview of person X

In [ ]:
interview_gt = pickle.load(interview_gt, open('/app/data/interview_gt.pkl', 'wb')) # might need to separate for each person
interviews, _, _, _ = load_interview("Newt Gingrich", suffix='10y2')

In [ ]:
interview_intrvllists = {}
gt_intrvllists = {}
for video_id, values in interview_gt.items():
    interview_intrvllists[video_id] = interviews.get_intervallist(video_id)
    gt_intrvllists[video_id] = IntervalList(values)
print_statistics(interview_intrvllists, gt_intrvllists)

## Pickle all computed interviews into one file

In [ ]:
interview_10y = {}
for person_name in guest_list:
    print(person_name)
    path = '/app/result/interview/{}-interview-10y2.pkl'.format(person_name.lower())
    interview_10y[person_name] = pickle.load(open(path, 'rb'))

In [ ]:
pickle.dump(interview_10y, open('/app/result/interview/interview_10y-all.pkl', 'wb'))